# Evaluate and fit a ML model on the EC flux tower data 



## Load modules

In [ ]:
import os
import shap
import xarray as xr
import numpy as np
import pandas as pd
import geopandas as gpd
import seaborn as sb
from joblib import dump
import multiprocessing
from pprint import pprint
import matplotlib.pyplot as plt
import statsmodels.api as sm
# from lightgbm import LGBMRegressor
from sklearn.ensemble import RandomForestRegressor
import lightgbm as lgbm
from sklearn.model_selection import LeaveOneGroupOut, GridSearchCV
from sklearn.metrics import mean_squared_error, mean_absolute_error

import sys
sys.path.append('/g/data/os22/chad_tmp/dea-notebooks/Tools')
from dea_tools.classification import spatial_clusters

import warnings
warnings.filterwarnings("ignore")

## Analysis Parameters

In [ ]:
model_name = 'AUS'
model_var = 'NEE'

scoring='neg_mean_absolute_error'
cluster_method = 'Hierarchical'
max_distance = 0.0000001 # map degrees

In [ ]:
ncpus=multiprocessing.cpu_count()
print('ncpus = '+str(ncpus))

## Prepare Data

In [ ]:
base = '/g/data/os22/chad_tmp/NEE_modelling/results/training_data/'
sites = os.listdir('/g/data/os22/chad_tmp/NEE_modelling/results/training_data/')

td = []
for site in sites:
    if '.csv' in site:
        xx = pd.read_csv(base+site, index_col='time', parse_dates=True)
        xx['site'] = site[0:5]
        td.append(xx)

ts = pd.concat(td).dropna()

## Modelling

In [ ]:
variables = [
            #'LAI_anom_RS',
             'kNDVI_anom_RS',
             'FPAR_RS',
             'LST_RS',
             'tree_cover_RS',
             'nontree_cover_RS',
             'nonveg_cover_RS',
             'LST-Tair_RS',
             'TWI_RS',
             'NDWI_RS',
             'rain_anom_RS',
             'rain_cml3_anom_RS',
             'rain_cml6_anom_RS',
             'rain_cml12_anom_RS',
             'srad_anom_RS',
             'vpd_RS',
             'tavg_anom_RS',
             'SOC_RS',
             #'CO2_RS',
             #'site'
            ]

In [ ]:
xx = []
yy = []
cc = []
dfs = []
for t in td:
    # t = t.drop('PFT_RS', axis=1)
    t = t.dropna()  # remove NaNS
    coords = t[['x_coord','y_coord']] #extract coords
    t = t.drop(['x_coord','y_coord','site'] , axis=1) # drop coords and Landcover
    df = t.drop(['NEE_SOLO_EC','GPP_SOLO_EC','ER_SOLO_EC'], axis=1) # seperate carbon fluxes
    
    #df = df.filter(regex='RS') # only use remote sensing variables   
    df = df[variables]

    if model_var == 'ET':
        df_var=t[model_var+'_EC']
    else:
        df_var=t[model_var+'_SOLO_EC'] # seperate out the variable we're modelling

    x = df.reset_index(drop=True)#.to_numpy()
    y = df_var.reset_index(drop=True)#.to_numpy()
    c = coords.reset_index(drop=True)
    xx.append(x)
    yy.append(y)
    cc.append(c)
    
x = np.concatenate([x for x in xx])
y = np.concatenate([y for y in yy])
c = np.concatenate([c for c in cc])

print(x.shape)

## Test model robustness with nested spatial leave-one-group-out cross validation

### Visualising spatial groupings

After setting the parameters, lets first generate spatial clusters using the DE Africa function spatial_clusters to visualize how our data will be grouped when running SKCV in the next step. You may want to refine the parameters to achieve a grouping that works for your dataset by resetting the parameters above


In [ ]:
aus=gpd.read_file('/g/data/os22/chad_tmp/NEE_modelling/data/aus_outline/AUS_2021_AUST_GDA2020.shp')

In [ ]:
#create clustes
spatial_groups = spatial_clusters(coordinates=c,
                                  method=cluster_method,
                                  max_distance=max_distance,
                                  n_groups=None,
                                  verbose=True)

#plot
fig, ax = plt.subplots(1,1, figsize=(10,6))
aus[aus.AUS_CODE21=='AUS'].plot(ax=ax, color='lightgrey')
ax.scatter(c[:, 0], c[:, 1], c=spatial_groups,
            s=50, cmap='tab20');
ax.set_xlim(110,155)
plt.title('Spatial clusters of training data');

In [ ]:
# create outer k-fold splits
outer_cv = LeaveOneGroupOut()

acc = []
rmse=[]
r2=[]
i = 1
for train_index, test_index in outer_cv.split(x, y, spatial_groups):
    print(f"Working on {i}/{len(np.unique(spatial_groups))} split", end='\r')
    
    model = RandomForestRegressor()
    # model = LGBMRegressor(random_state=1, n_jobs=ncpus)

    # index training, testing, and coordinate data
    X_tr, X_tt = x[train_index, :], x[test_index, :]
    y_tr, y_tt = y[train_index], y[test_index]
    inner_groups = spatial_groups[train_index]
    
    # inner split on data within outer split
    # inner_cv = LeaveOneGroupOut()
    
    #print(inner_cv.get_n_splits(X_tr, y_tr, inner_groups))
#     clf = GridSearchCV(
#         estimator=model,
#         param_grid=param_grid,
#         scoring=scoring,
#         n_jobs=ncpus,
#         refit=True,
#         cv=inner_cv.split(X_tr, y_tr, inner_groups),
#     )

#     clf.fit(X_tr, y_tr)
    
#     # predict using the best model
#     best_model = clf.best_estimator_
#     pred = best_model.predict(X_tt)
    
    model = model.fit(X_tr, y_tr)
    pred = model.predict(X_tt)

    # evaluate model w/ multiple metrics
    X_ = sm.add_constant(y_tt)
    model = sm.OLS(pred,X_)
    r2_ = model.fit().rsquared
    r2.append(r2_)
    ac = mean_absolute_error(y_tt, pred)
    acc.append(ac)
    rmse_ = np.sqrt(mean_squared_error(y_tt, pred))
    rmse.append(rmse_)
    
    #Compare time-series in a plot
    df_y_true = ts.iloc[test_index]
    df_y_true['pred_'+model_var] = pred 
    sb.set_style("darkgrid")
    fig,ax = plt.subplots(1,1, figsize=(12,5))
    sb.lineplot(x='time', y=model_var+'_SOLO_EC', data=df_y_true, color="green", linewidth=1, ax=ax)
    sb.lineplot(x='time', y='pred_'+model_var, data=df_y_true, color="blue", linewidth=1, ax=ax)

    ax.text(.015, .95, 'r = {:.3f}'.format(r2_),
            transform=ax.transAxes)
    ax.text(.015, .9, 'mae = {:.3g}'.format(ac),
            transform=ax.transAxes)

    ax.set_ylabel(model_var)
    ax.set_xlabel(None)
    ax.set_title(df_y_true['site'][0]+': '+model_var + ' Cross Validation Results')
    plt.legend(labels=["True","Prediction"], loc='lower left')
    plt.tight_layout()
    fig.savefig("/g/data/os22/chad_tmp/NEE_modelling/results/cross_validation/"+df_y_true['site'][0]+"_"+model_var+".png")
    plt.close()
    
    i += 1

In [ ]:
print("Mean MAE accuracy: "+ str(round(np.mean(acc), 2)))
print("Std dev of MAE accuracy: "+ str(round(np.std(acc), 2)))
print('\n')
print("Mean RMSE: "+ str(round(np.mean(rmse), 2)))
print("Std dev RMSE: "+ str(round(np.std(rmse), 2)))
print('\n')
print("Mean r2: "+ str(round(np.mean(r2), 2)))
print("Std dev r2: "+ str(round(np.std(r2), 2)))

In [ ]:
print(np.min(r2))

In [ ]:
print(np.max(r2))

## Optimize hyperparameters

In [ ]:
# Create the parameter grid based
param_grid = {
    'num_leaves': [7, 14, 21, 28, 31, 50],
    'min_data_in_leaf':[15, 20, 30],
    #'boosting_type ': ['gbdt', 'dart'],
    'max_depth': [3, 5, 10, 20],
    'n_estimators': [100, 200, 300, 400],
}

In [ ]:
%%time
#generate n_splits of train-test_split
logo = LeaveOneGroupOut()

#instatiate a gridsearchCV
clf = GridSearchCV(LGBMRegressor(),
                   param_grid,
                   scoring=scoring,
                   verbose=1,
                   cv=logo.split(x, y, spatial_groups),
                   n_jobs=ncpus)

clf.fit(x, y)

In [ ]:
print("The most accurate combination of tested parameters is: ")
pprint(clf.best_params_)
print('\n')
print("The best score using these parameters is: ")
print(round(clf.best_score_*-1, 2))

## Fit on all data using best params

In [ ]:
model = LGBMRegressor(**clf.best_params_, random_state=1, n_jobs=ncpus)
model.fit(x, y)

## Examine feature importance

SHAP (SHapley Additive exPlanations) is a game theoretic approach to explain the output of any machine learning model

https://github.com/slundberg/shap

https://towardsdatascience.com/explaining-scikit-learn-models-with-shap-61daff21b12a

In [ ]:
# explain the model's predictions using SHAP
explainer = shap.Explainer(model)
shap_values = explainer(x)

In [ ]:
#add names of features
shap_values.feature_names = df.columns.values

# visualize the importances
# shap.plots.waterfall(shap_values[0])
# shap.plots.beeswarm(shap_values)
shap.plots.bar(shap_values)

## Save the model

In [ ]:
dump(model, '/g/data/os22/chad_tmp/NEE_modelling/results/models/'+model_name+'_'+model_var+'_RF_LOGO_model.joblib')

## Predict on all the data

In [ ]:
y_pred = model.predict(x)

rmse = float(format(np.sqrt(mean_squared_error(y, y_pred)), '.3f'))
print("RMSE:", rmse);

In [ ]:
sb.scatterplot(x=y,y=y_pred,color="#338844", edgecolor="white", s=50, lw=1, alpha=0.5)
sb.regplot(x=y, y=y_pred, scatter=False, color='m')
sb.regplot(x=y, y=y, color='black', scatter=False, line_kws={'linestyle':'dashed'});
plt.xlabel('True')
plt.ylabel('Prediction');